
# Link Prediction in Social Networks

In this notebook, we aim to predict links in a social network using node attributes and graph structures. We'll utilize a dataset representing a network, remove some links to create a training set, and then try to predict the removed links based on various features.

---


In [1]:
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities

import numpy as np
import matplotlib.pyplot as plt
import math
from collections import Counter

from tqdm import tqdm
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree, DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score,roc_curve, auc, confusion_matrix


In [ ]:
# We will start by importing the network from an edgelist

G1 = nx.read_edgelist("edges_train.edgelist", data=False, nodetype = int, delimiter=',') # import
pos = nx.spring_layout(G1)
layout = nx.spring_layout(G1, k=0.4)
plt.figure(figsize=(10, 8))
nx.draw(G1, pos=layout, with_labels=True,  node_size=20, font_size=6,  edge_color='lightgray',  # Edge color
    width=1.0,          # Edge width
               # Edge transparency (alpha))
       )

In [ ]:
# Load node attributes from CSV into a DataFrame
node_attributes_file = 'attributes.csv'
node_attributes_df = pd.read_csv(node_attributes_file)

# Create a dictionary with node IDs as keys and attributes as values
node_attributes = dict(zip(node_attributes_df['ID'], node_attributes_df['attribute']))

# Assign attributes to nodes in the graph
for node_id, attributes in node_attributes.items():
    if G1.has_node(node_id):
        G1.nodes[node_id]['attribute'] = attributes
    else:
        print(f"Node {node_id} from the attribute file does not exist in the edge list.")


In [4]:
# Extract unique categories from the node attributes
categories = np.unique(list(node_attributes.values())).reshape(-1, 1)

# One-hot encode the categories
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
onehot_encoded = encoder.fit_transform(categories)

# Create a mapping of category to its one-hot encoding
category_to_onehot = {category: onehot_encoded[i] for i, category in enumerate(encoder.categories_[0])}

# One-hot encode the node attributes
onehot_node_attributes = {node: category_to_onehot[attr] for node, attr in node_attributes.items()}

print(f"Original node attribute: {node_attributes[0]} \nBecomes a one-hot encoded node attribute: {onehot_node_attributes[0]}")
print(f"Original node attribute: {node_attributes[33]} \nBecomes a one-hot encoded node attribute: {onehot_node_attributes[33]}")
node_features = onehot_node_attributes

Original node attribute: c 
Becomes a one-hot encoded node attribute: [1. 0. 0. 0. 0.]
Original node attribute: d 
Becomes a one-hot encoded node attribute: [0. 1. 0. 0. 0.]


/opt/homebrew/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:

def calculate_jaccard_coefficient(G, u, v):
    """
    Calculate the Jaccard coefficient for nodes u and v in the given graph.
    """
    common_neighbors = len(list(nx.common_neighbors(G, u, v)))
    union_neighbors = len(set(G.neighbors(u)).union(G.neighbors(v)))
    
    if union_neighbors == 0:
        return 0.0  # Return 0 if no neighbors to calculate Jaccard
    
    jaccard_coefficient = common_neighbors / union_neighbors
    return jaccard_coefficient

from math import log10
def intersection(list1, list2):
    inter_list = []
    for value in list1:
        if value in list2:
            inter_list.append(value)
    return inter_list


def common_neighbors(graph, n1, n2):
    ngh1 = list(graph.neighbors(n1))
    ngh2 = list(graph.neighbors(n2))

    if n1 in ngh2:                      
        ngh1.remove(n2)                  
        ngh2.remove(n1)                  
                                        
    return len(intersection(ngh1, ngh2))

def jaccards_coefficient(graph, n1, n2):
    score = 0
    ngh1 = list(graph.neighbors(n1))
    ngh2 = list(graph.neighbors(n2))

    if n1 in ngh2:   
        ngh1.remove(n2)                 
        ngh2.remove(n1)

    try:
        score = len(intersection(ngh1, ngh2))/len(ngh1 + ngh2)
    except ZeroDivisionError:
        score = 0

    return score


def preferential_attachment(graph, n1, n2):
    ngh1 = list(graph.neighbors(n1))
    ngh2 = list(graph.neighbors(n2))

    if n1 in ngh2:
        ngh1.remove(n2)
        ngh2.remove(n1)

    return len(ngh1) * len(ngh2)


def adamic_adar(graph, n1, n2):
    score = 0
    ngh1 = list(graph.neighbors(n1))
    ngh2 = list(graph.neighbors(n2))

    if n1 in ngh2:
        ngh1.remove(n2)
        ngh2.remove(n1)

    inter_list = intersection(ngh1, ngh2)

    for node in inter_list:
        try:
            score += 1/log10(len(list(graph.neighbors(node))))
        except ZeroDivisionError:
            continue
 
    return score


def resource_allocation(graph, n1, n2):
    score = 0
    ngh1 = list(graph.neighbors(n1))
    ngh2 = list(graph.neighbors(n2))

    if n1 in ngh2:
        ngh1.remove(n2)
        ngh2.remove(n1)

    inter_list = intersection(ngh1, ngh2)

    for node in inter_list:
        try:
            score += 1/len(list(graph.neighbors(node)))
        except ZeroDivisionError:
            continue

    return score

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
triangles = nx.triangles(G1)
betweenness_centrality = nx.betweenness_centrality(G1)
short_paths = dict(nx.shortest_path_length(G1)) 

# Define the edge_features function to be executed in parallel
def calculate_edge_features(u, v):
   
    jaccard_coefficient = calculate_jaccard_coefficient(G1, u, v)
    jaccard_coeff2 = jaccards_coefficient(G1, u, v)
    
    
        # Sample degree-related features
    degree_u = np.array([G1.degree()[u]])  # Wrap in np.array to create a 2D array
    degree_v = np.array([G1.degree()[v]])
    degree_sum = np.array([G1.degree()[u] + G1.degree()[v]])
    degree_diff = np.array([abs(G1.degree()[u] - G1.degree()[v])])

    # Create a Min-Max scaler
    scaler = MinMaxScaler()

    # Scale the features
    degree_u_scaled = scaler.fit_transform(degree_u.reshape(-1, 1))[0][0]
    degree_v_scaled = scaler.fit_transform(degree_v.reshape(-1, 1))[0][0]
    degree_sum_scaled = scaler.fit_transform(degree_sum.reshape(-1, 1))[0][0]
    degree_diff_scaled = scaler.fit_transform(degree_diff.reshape(-1, 1))[0][0]
    
#     print(degree_v)
    
#     if jaccard_coefficient != jaccard_coeff2:
#         print(jaccard_coefficient, jaccard_coeff2)

    if jaccard_coeff2 <= 0:
        jaccard_log = 0.0
    else:
        jaccard_log = np.log(jaccard_coeff2)

    # Return the edge features
    return np.concatenate((
        np.concatenate((node_features[u], node_features[v])),
        [
            G1.degree()[u],
            G1.degree()[v],
            G1.degree()[u] + G1.degree()[v],
            abs(G1.degree()[u] - G1.degree()[v]),
            preferential_attachment(G1, u, v),
            common_neighbors(G1, u, v),
#             jaccard_coefficient,
            jaccard_coeff2,
            jaccard_log,
            adamic_adar(G1, u, v),
            resource_allocation(G1, u, v),
            degree_u_scaled,
            degree_v_scaled,
            degree_sum_scaled,
            degree_diff_scaled
        ])
    )



In [6]:
# Let us now create a training set where X will correspond to features for each possible edge and Y the predciton
X = []
Y = []
N = G1.number_of_nodes() 
M = int(G1.number_of_edges()*.5)

# Add positive examples where edge exist
for (i, j) in tqdm(G1.edges):
    X.append(calculate_edge_features(i, j))
    Y.append(1)

    

# Add positive examples where edge does not exist
# Reflect: how many negative examples should you include?
for kk in tqdm(range(M)):
        i = np.random.randint(N)
        j = np.random.randint(N)

        while G1.has_edge(i,j) or i == j:
            i = np.random.randint(N)
            j = np.random.randint(N)

        X.append(calculate_edge_features(i, j))
        Y.append(0)

100%|███████████████████████████████████████████| 2184/2184 [00:00<00:00, 3411.80it/s]


In [7]:
#  Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5,random_state=42, shuffle=True)


In [8]:
print(f"Total edges: {len(y_train)} | positive: {sum(y_train)} | negative: {len(y_train) - sum(y_train) } | X {len(X)}")

Total edges: 3931 | positive: 2599 | negative: 1332 | X 6552


In [9]:
param_grid = {
    'criterion': ['gini', 'entropy'],          # Criterion for split (Gini impurity or entropy)
    'max_depth': [None, 2,3,4 ,5, 6, 7, 8,9, 10],   # Maximum depth of the tree
    'min_samples_split': [2,3, 4,5,6,7],            # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],               # Minimum samples required at a leaf node
    'max_features': ['sqrt', 'log2']    # Number of features to consider for the best split
}


# # Create the GridSearchCV object
gs_dt = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=param_grid,
    scoring='accuracy',  # Choose appropriate scoring metric
    cv=10,                # Number of cross-validation fold
)

gs_dt.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 3, 4, 5, 6, 7]},
             scoring='accuracy')

In [10]:
decision_tree = gs_dt.best_estimator_
# print(decision_tree)
decision_tree = DecisionTreeClassifier(max_depth=7, criterion='entropy', max_features='sqrt')
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=7, max_features='sqrt')

In [11]:
param_grid = {
    'n_estimators': [ 55,  65, 70,  80],  # Number of trees in the forest
    'criterion': ['entropy'],  # Criterion for information gain calculation
    'max_depth': [    25, 40, 50,  70],  # Maximum depth of the tree
#     'min_samples_split': [2, 3,  4, 5, 6, 7],  # Minimum samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],  # Minimum samples required at the leaf node
    'max_features': ['sqrt', 'log2'],
    'class_weight': ['balanced'] # Number of features to consider for best split
}

gs_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=10, n_jobs=-1)
gs_rf.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced'], 'criterion': ['entropy'],
                         'max_depth': [25, 40, 50, 70],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [55, 65, 70, 80]})

In [12]:
random_forest = gs_rf.best_estimator_
# print(random_forest)
random_forest.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=40, max_features='log2', n_estimators=80)

In [14]:
from sklearn.linear_model import LogisticRegression

# Parameter grid for Logistic Regression
param_grid_logreg = {
    'penalty': ['l2'],  # Penalty norm
    'C': [4,5, 10,11, 12, 13, 14, 15, 20],  # Inverse of regularization strength
    'solver': ['liblinear', 'saga', 'newton-cholesky', 'newton-cg'],  # Solver algorithm
    'class_weight': ['balanced']  # Class weights
}

# Grid search for Logistic Regression
gs_logreg = GridSearchCV(estimator=LogisticRegression(max_iter=10000, solver='newton-cholesky'), param_grid=param_grid_logreg, cv=10, n_jobs=-1)
gs_logreg.fit(X_train, y_train)
logreg = gs_logreg.best_estimator_
logreg.fit(X_train, y_train)


LogisticRegression(C=20, class_weight='balanced', max_iter=10000,
                   solver='liblinear')

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid_gb = {
    'n_estimators': [ 190, 200,210, 220],  # Number of boosting stages to be run
    'learning_rate': [ .5, .6, .7],  # Step size shrinkage
    'max_depth': [ 8, 15, 20, 30, 50],  # Maximum depth of the individual estimators
    'subsample': [ 0.7, .8, .9] ,
    'ccp_alpha': [0.005, 0.006, 0.007, 0.008, 0.009]# Fraction of samples used for fitting the trees
}

gs_gb = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=param_grid_gb, cv=5, n_jobs=-1)
gs_gb.fit(X_train, y_train)
gb = gs_gb.best_estimator_

KeyboardInterrupt: 

In [16]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np

# Define a function to print evaluation metrics
def print_evaluation_metrics(model_name, metrics):
    print("=== {} ===".format(model_name))
    print("Precision:", np.mean(metrics['precision']))
    print("Recall:", np.mean(metrics['recall']))
    print("F1 Score:", np.mean(metrics['f1']))
    print("Accuracy:", np.mean(metrics['accuracy']))
    print("Variance of Precision:", np.std(metrics['precision']))
    print("Variance of Recall:", np.std(metrics['recall']))
    print("Variance of F1 Score:", np.std(metrics['f1']))
    print("Variance of Accuracy:", np.std(metrics['accuracy']))
    print("================")

# Set up 10-fold cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate Decision Tree model using cross-validation
def compute_metrics(model):
    return {
    'precision': cross_val_score(model, X_val, y_val, cv=cv, scoring='precision'),
    'recall': cross_val_score(model, X_val, y_val, cv=cv, scoring='recall'),
    'f1': cross_val_score(model, X_val, y_val, cv=cv, scoring='f1'),
    'accuracy': cross_val_score(model, X_val, y_val, cv=cv, scoring='accuracy')
}



for model in [decision_tree, random_forest, gb, logreg]:
    print_evaluation_metrics(f"Cross-validated {type(model).__name__}", compute_metrics(model))


NameError: name 'gb' is not defined

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Define a function to print evaluation metrics for the test set
def print_test_evaluation_metrics(model_name, y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    
    print("=== {} ===".format(model_name))
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
    print("================")

# Assuming decision_tree is the trained model
# Evaluate Decision Tree model on test set
for model in [decision_tree, random_forest, gb, logreg]:

    y_pred_test = model.predict(X_test)

    # Print evaluation metrics for the test set
    print_test_evaluation_metrics(f"{type(model).__name__} (Test)", y_test, y_pred_test)


NameError: name 'gb' is not defined

In [18]:

# Assuming clf1 and clf2 are your classifiers and X_test, y_test are your test data
# Calculate ROC curve and AUC for clf1
model1 = random_forest
model2 = decision_tree
model3 = gb
model4 = logreg

fpr1, tpr1, thresholds1 = roc_curve(y_test, model1.predict_proba(X_test)[:, 1])
roc_auc1 = auc(fpr1, tpr1)

# Calculate ROC curve and AUC for clf2
fpr2, tpr2, thresholds2 = roc_curve(y_test, model2.predict_proba(X_test)[:, 1])
roc_auc2 = auc(fpr2, tpr2)

# Calculate ROC curve and AUC for clf2
fpr3, tpr3, thresholds2 = roc_curve(y_test, model3.predict_proba(X_test)[:, 1])
roc_auc3 = auc(fpr3, tpr3)

# Calculate ROC curve and AUC for clf2
fpr4, tpr4, thresholds4 = roc_curve(y_test, model4.predict_proba(X_test)[:, 1])
roc_auc4 = auc(fpr4, tpr4)

# Plot ROC curves for both classifiers in one plot
plt.figure(figsize=(10, 6))

# Plot ROC curve for clf1


# Plot ROC curve for clf2
plt.plot(fpr4, tpr4, color='purple', lw=2, label='ROC curve for {} (area = {:.2f})'.format('logistic regression', roc_auc4))

plt.plot(fpr2, tpr2, color='blue', lw=2, label='ROC curve for {} (area = {:.2f})'.format('decision tree', roc_auc2))
plt.plot(fpr1, tpr1, color='darkorange', lw=2, label='ROC curve for {} (area = {:.2f})'.format('random forest', roc_auc1))
# Plot ROC curve for clf2
plt.plot(fpr3, tpr3, color='green', lw=2, label='ROC curve for {} (area = {:.2f})'.format('gradient boosting', roc_auc3))


plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', size=20)
plt.ylabel('True Positive Rate', size=20)
plt.title('ROC Curves for Decision Tree, Random Forest & Gradient Boosting')
plt.legend(loc='lower right', prop={'size': 14})
# plt.savefig('roc_plot.png')
plt.show()

NameError: name 'gb' is not defined